In [1]:
%matplotlib notebook

import numpy as np
import rasterio as rio
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterstats import zonal_stats

plt.rcParams.update({'font.size': 22}) # update the font size for our plots to be size 22

In [2]:
def generate_handles(labels, colors, edge='k', alpha=1):
    lc = len(colors)  # get the length of the color list
    handles = []
    for i in range(len(labels)):
        handles.append(mpatches.Rectangle((0, 0), 1, 1, facecolor=colors[i % lc], edgecolor=edge, alpha=alpha))
    return handles

In [3]:
# open the land cover raster and read the data
with rio.open('data_files/LCM2015.tif') as dataset:
    xmin, ymin, xmax, ymax = dataset.bounds 
    crs = dataset.crs
    lcm = dataset.read(1)
    affine_tfm = dataset.transform

# now, load the county dataset from the week 2 folder
radon = gpd.read_file('data_files/radonNI.shp').to_crs(crs)

In [4]:
def count_unique(array, nodata=0):
    '''
    Count the unique elements of an array.

    :param array: Input array
    :param nodata: nodata value to ignore in the counting
    
    :returns count_dict: a dictionary of unique values and counts
    '''
    count_dict = {}
    for val in np.unique(array):
        if val == nodata:
            continue
        count_dict[str(val)] = np.count_nonzero(array == val)
    return count_dict

unique_lcm = count_unique(lcm)
print(unique_lcm)

{'1': 91, '2': 693, '3': 498, '4': 9808, '5': 93, '6': 1, '7': 377, '9': 144, '10': 487, '11': 834, '12': 1, '13': 33, '14': 86, '15': 2, '16': 26, '17': 2, '18': 14, '20': 48, '21': 443, '127': 12094}


In [7]:
lcm_stats = zonal_stats(radon, lcm, affine=affine_tfm, categorical=True, nodata=0)

print(lcm_stats[5])

{10.0: 1}
